# Week 14: Advanced API Integration and Final Project
**고급 API 연동 및 종합 프로젝트**

**Duration (수업 시간)**: 3 hours (3시간)  
**Structure (구성)**: Lecture & Lab 2 hours + Quiz 1 hour (강의 및 실습 2시간 + 퀴즈 1시간)  
**Level (수준)**: Intermediate (중급)

---

## Learning Objectives (학습 목표)

By the end of this lesson, students will be able to:
이 수업을 마친 후 학생들은 다음을 할 수 있습니다:

- Combine data from different sources (여러 소스의 데이터 결합)
- Handle errors when things go wrong (문제가 발생했을 때 오류 처리)
- Create a simple dashboard (간단한 대시보드 생성)
- Save data to avoid repeated work (반복 작업을 피하기 위해 데이터 저장)

---

## 1. Multiple API Integration (다중 API 통합)

### What does "integration" mean? (통합이 무엇을 의미하나요?)

Integration means combining different pieces of information. Think of making a sandwich - you combine bread, meat, and vegetables to make something better.
통합은 서로 다른 정보 조각들을 결합하는 것을 의미합니다. 샌드위치 만들기를 생각해보세요 - 빵, 고기, 야채를 결합해서 더 좋은 것을 만듭니다.

### Step 1: Create Simple Data Sources (1단계: 간단한 데이터 소스 생성)

Let's start with two simple functions that give us different information:
서로 다른 정보를 제공하는 두 개의 간단한 함수로 시작해봅시다:

In [ ]:
def get_weather(city):
    # This is like one API that gives weather info
    weather_info = {
        "Seoul": "Sunny, 25°C",
        "Tokyo": "Rainy, 18°C"
    }
    return weather_info.get(city, "No weather data")

def get_population(city):
    # This is like another API that gives population info
    population_info = {
        "Seoul": "9.7 million people",
        "Tokyo": "13.9 million people"
    }
    return population_info.get(city, "No population data")

# Test each function separately
print("Weather:", get_weather("Seoul"))
print("Population:", get_population("Seoul"))

### Step 2: Combine the Information (2단계: 정보 결합)

Now let's create a function that uses both data sources:
이제 두 데이터 소스를 모두 사용하는 함수를 만들어봅시다:

In [ ]:
def get_city_info(city):
    # Get information from both sources
    weather = get_weather(city)
    population = get_population(city)
    
    # Combine them into one result
    city_summary = {
        "city": city,
        "weather": weather,
        "population": population
    }
    
    return city_summary

# Test the combined function
seoul_info = get_city_info("Seoul")
print(f"City: {seoul_info['city']}")
print(f"Weather: {seoul_info['weather']}")
print(f"Population: {seoul_info['population']}")

**Why is this useful?** Now you can get all city information with one function call instead of calling multiple functions separately.
**왜 이것이 유용한가요?** 이제 여러 함수를 따로 호출하는 대신 하나의 함수 호출로 모든 도시 정보를 얻을 수 있습니다.

---

## 2. Error Handling for APIs (API 오류 처리)

### Why do we need error handling? (왜 오류 처리가 필요한가요?)

Sometimes things go wrong:
때때로 문제가 발생합니다:
- Internet connection fails (인터넷 연결 실패)
- API is temporarily down (API가 일시적으로 다운)
- We make a mistake in our code (코드에서 실수를 함)

Without error handling, your program crashes. With error handling, it keeps working.
오류 처리 없이는 프로그램이 충돌합니다. 오류 처리가 있으면 계속 작동합니다.

### Step 1: See What Happens Without Error Handling (1단계: 오류 처리 없이 무엇이 일어나는지 보기)

In [ ]:
def risky_function(city):
    # This function might fail
    if city == "BadCity":
        raise Exception("This city doesn't exist!")
    return f"Information for {city}"

# This will work fine
print(risky_function("Seoul"))

# This will crash the program (don't run this yet!)
# print(risky_function("BadCity"))

### Step 2: Add Error Handling (2단계: 오류 처리 추가)

In [ ]:
def safe_function(city):
    try:
        # Try to get the information
        result = risky_function(city)
        return result
    except Exception as error:
        # If something goes wrong, handle it gracefully
        return f"Sorry, couldn't get info for {city}. Error: {error}"

# Test both cases
print(safe_function("Seoul"))      # This works
print(safe_function("BadCity"))    # This handles the error

**What happened?** The `try-except` block caught the error and gave us a nice message instead of crashing.
**무엇이 일어났나요?** `try-except` 블록이 오류를 잡아서 충돌하는 대신 친절한 메시지를 주었습니다.

### Step 3: Apply Error Handling to Our API Integration (3단계: API 통합에 오류 처리 적용)

In [ ]:
def safe_city_info(city):
    try:
        # Try to get weather info
        weather = get_weather(city)
        
        # Try to get population info
        population = get_population(city)
        
        # If both work, return combined info
        return {
            "city": city,
            "weather": weather,
            "population": population,
            "status": "success"
        }
    except Exception as error:
        # If anything fails, return error info
        return {
            "city": city,
            "error": str(error),
            "status": "failed"
        }

# Test it
result = safe_city_info("Seoul")
if result["status"] == "success":
    print(f"Got info for {result['city']}")
else:
    print(f"Failed to get info: {result['error']}")

---

## 3. Data Storage and Caching (데이터 저장 및 캐싱)

### Why save data? (왜 데이터를 저장하나요?)

Imagine you ask someone the same question 100 times. That's annoying, right? APIs feel the same way. If we save the answer the first time, we don't need to ask again.
누군가에게 같은 질문을 100번 한다고 상상해보세요. 짜증나겠죠? API도 같은 느낌입니다. 처음에 답을 저장하면 다시 물어볼 필요가 없습니다.

### Step 1: Simple Data Saving (1단계: 간단한 데이터 저장)

In [ ]:
# We'll use a simple dictionary to store data in memory
data_cache = {}

def get_and_save_weather(city):
    # Check if we already have the data
    if city in data_cache:
        print(f"Using saved data for {city}")
        return data_cache[city]
    
    # If not, get new data
    print(f"Getting new data for {city}")
    weather = get_weather(city)
    
    # Save it for next time
    data_cache[city] = weather
    
    return weather

# Test it - first call gets new data
print(get_and_save_weather("Seoul"))

# Second call uses saved data
print(get_and_save_weather("Seoul"))

### Step 2: Save to File (2단계: 파일에 저장)

In [ ]:
import json

def save_to_file(data, filename):
    # Convert Python data to JSON and save to file
    with open(filename, 'w') as file:
        json.dump(data, file)
    print(f"Data saved to {filename}")

def load_from_file(filename):
    # Try to load data from file
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
        print(f"Data loaded from {filename}")
        return data
    except FileNotFoundError:
        print(f"No saved file found: {filename}")
        return None

# Example usage
city_data = {"Seoul": "Sunny, 25°C", "Tokyo": "Rainy, 18°C"}

# Save data
save_to_file(city_data, "weather_data.json")

# Load data
loaded_data = load_from_file("weather_data.json")
print("Loaded data:", loaded_data)

---

## 4. Creating Simple Dashboards (간단한 대시보드 생성)

### What is a dashboard? (대시보드란 무엇인가요?)

A dashboard is like the control panel in a car - it shows you important information all in one place.
대시보드는 자동차의 계기판과 같습니다 - 중요한 정보를 모두 한 곳에서 보여줍니다.

### Step 1: Simple Information Display (1단계: 간단한 정보 표시)

In [ ]:
def simple_dashboard(city):
    print("=" * 30)
    print(f"   DASHBOARD FOR {city}")
    print("=" * 30)
    
    # Get and display weather
    weather = get_weather(city)
    print(f"Weather: {weather}")
    
    # Get and display population
    population = get_population(city)
    print(f"Population: {population}")
    
    print("=" * 30)

# Test the dashboard
simple_dashboard("Seoul")

### Step 2: Add More Information (2단계: 더 많은 정보 추가)

In [ ]:
from datetime import datetime

def enhanced_dashboard(city):
    # Header
    print("\n" + "=" * 40)
    print(f"      CITY DASHBOARD: {city.upper()}")
    print("=" * 40)
    
    # Current time
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M")
    print(f"Updated: {current_time}")
    print()
    
    # Weather section
    print("WEATHER:")
    weather = get_weather(city)
    print(f"  Current: {weather}")
    print()
    
    # Population section
    print("DEMOGRAPHICS:")
    population = get_population(city)
    print(f"  Population: {population}")
    print()
    
    # Footer
    print("=" * 40)

# Test enhanced dashboard
enhanced_dashboard("Seoul")

---

## Lab Exercises (실습)

### Lab 1: Personal Daily Briefing (개인 일일 브리핑)

**Problem**: Create a morning briefing that combines weather, news, and your schedule.
문제: 날씨, 뉴스, 일정을 결합한 아침 브리핑을 만드세요.

**Let's build this step by step:** (단계별로 만들어봅시다:)

**Step 1: Create data sources** (1단계: 데이터 소스 생성)

In [ ]:
def get_weather_simple(city):
    weather = {
        "Seoul": "Sunny, 22°C",
        "Busan": "Cloudy, 25°C"
    }
    return weather.get(city, "Weather not available")

def get_news_simple():
    return [
        "Tech conference starts today",
        "New subway line opens",
        "Weekend festival announced"
    ]

def get_schedule_simple():
    return [
        "9:00 AM - Team meeting",
        "2:00 PM - Project review",
        "5:00 PM - Gym workout"
    ]

**Step 2: Combine everything** (2단계: 모든 것 결합)

In [ ]:
def daily_briefing(city):
    print("\n" + "="*50)
    print("           GOOD MORNING!")
    print("           Daily Briefing")
    print("="*50)
    
    # Show current date
    from datetime import datetime
    today = datetime.now().strftime("%A, %B %d, %Y")
    print(f"Today is: {today}")
    print()
    
    # Weather section
    print("WEATHER:")
    weather = get_weather_simple(city)
    print(f"  {weather}")
    print()
    
    # News section
    print("TODAY'S NEWS:")
    news = get_news_simple()
    for i, headline in enumerate(news, 1):
        print(f"  {i}. {headline}")
    print()
    
    # Schedule section
    print("YOUR SCHEDULE:")
    schedule = get_schedule_simple()
    for item in schedule:
        print(f"  • {item}")
    
    print("\n" + "="*50)
    print("Have a great day!")

# Test the briefing
daily_briefing("Seoul")

### Lab 2: Simple Stock Tracker (간단한 주식 추적기)

**Problem**: Create a stock price tracker that shows current prices.
문제: 현재 가격을 보여주는 주식 가격 추적기를 만드세요.

**Step 1: Create stock data** (1단계: 주식 데이터 생성)

In [ ]:
def get_stock_price(symbol):
    # Simulated stock prices
    stocks = {
        "AAPL": 175.50,
        "GOOGL": 2750.80,
        "TSLA": 245.60
    }
    return stocks.get(symbol, None)

def get_stock_change(symbol):
    # Simulated price changes
    changes = {
        "AAPL": +2.30,
        "GOOGL": -15.20,
        "TSLA": +8.90
    }
    return changes.get(symbol, 0)

**Step 2: Create the tracker** (2단계: 추적기 생성)

In [ ]:
def stock_tracker(symbols):
    print("\n" + "="*40)
    print("        STOCK TRACKER")
    print("="*40)
    
    # Show current time
    from datetime import datetime
    update_time = datetime.now().strftime("%H:%M:%S")
    print(f"Last updated: {update_time}")
    print()
    
    # Show each stock
    for symbol in symbols:
        price = get_stock_price(symbol)
        change = get_stock_change(symbol)
        
        if price:
            # Format the change with + or - sign
            if change > 0:
                change_text = f"+${change:.2f}"
                trend = "📈"
            elif change < 0:
                change_text = f"${change:.2f}"
                trend = "📉"
            else:
                change_text = "$0.00"
                trend = "➡️"
            
            print(f"{symbol}: ${price:.2f} ({change_text}) {trend}")
        else:
            print(f"{symbol}: Price not available")
    
    print("="*40)

# Test the tracker
my_stocks = ["AAPL", "GOOGL", "TSLA"]
stock_tracker(my_stocks)

### Lab 3: Information Dashboard (정보 대시보드)

**Problem**: Create a dashboard that shows multiple types of information with error handling.
문제: 오류 처리와 함께 여러 유형의 정보를 보여주는 대시보드를 만드세요.

**Step 1: Create data functions with error handling** (1단계: 오류 처리가 있는 데이터 함수 생성)

In [ ]:
def safe_get_weather(city):
    try:
        weather_data = {
            "Seoul": "Sunny, 23°C",
            "Tokyo": "Rainy, 19°C"
        }
        return weather_data[city]
    except KeyError:
        return "Weather data not available"

def safe_get_traffic(city):
    try:
        traffic_data = {
            "Seoul": "Heavy traffic on Highway 1",
            "Tokyo": "Normal traffic conditions"
        }
        return traffic_data[city]
    except KeyError:
        return "Traffic data not available"

def safe_get_events(city):
    try:
        events_data = {
            "Seoul": ["Concert at Olympic Park", "Art exhibition downtown"],
            "Tokyo": ["Cherry blossom festival", "Tech conference"]
        }
        return events_data[city]
    except KeyError:
        return ["No events data available"]

**Step 2: Create comprehensive dashboard** (2단계: 종합 대시보드 생성)

In [ ]:
def comprehensive_dashboard(city):
    print("\n" + "="*50)
    print(f"       COMPREHENSIVE DASHBOARD")
    print(f"            {city.upper()}")
    print("="*50)
    
    # Current time
    from datetime import datetime
    now = datetime.now()
    print(f"Date: {now.strftime('%Y-%m-%d')}")
    print(f"Time: {now.strftime('%H:%M:%S')}")
    print()
    
    # Weather section
    print("🌤️  WEATHER:")
    weather = safe_get_weather(city)
    print(f"   {weather}")
    print()
    
    # Traffic section  
    print("🚗 TRAFFIC:")
    traffic = safe_get_traffic(city)
    print(f"   {traffic}")
    print()
    
    # Events section
    print("🎉 EVENTS:")
    events = safe_get_events(city)
    if isinstance(events, list):
        for event in events:
            print(f"   • {event}")
    else:
        print(f"   {events}")
    
    print("\n" + "="*50)
    print("Dashboard updated successfully!")

# Test with different cities
comprehensive_dashboard("Seoul")
print("\n" + "="*20 + " TESTING ERROR HANDLING " + "="*20)
comprehensive_dashboard("Unknown City")

---

## Quiz Section (퀴즈)

### Quiz 1: Simple Data Integration

**Question**: Create a function called `get_student_info(name)` that combines data from two sources: grades and attendance. Return a dictionary with both pieces of information.

두 소스의 데이터를 결합하는 `get_student_info(name)` 함수를 만드세요: 성적과 출석. 두 정보를 모두 포함하는 딕셔너리를 반환하세요.

In [ ]:
# Use this sample data:
grades = {"Alice": 85, "Bob": 92}
attendance = {"Alice": "95%", "Bob": "88%"}

**Write your answer here (답을 여기에 작성하세요)**:

In [ ]:
# Your code here

### Quiz 2: Error Handling Practice

**Question**: Write a function that safely divides two numbers. If there's an error (like dividing by zero), return an error message instead of crashing.

두 숫자를 안전하게 나누는 함수를 작성하세요. 오류가 있으면 (0으로 나누기 같은) 충돌하는 대신 오류 메시지를 반환하세요.

**Write your answer here (답을 여기에 작성하세요)**:

In [ ]:
# Your code here

### Quiz 3: Mini Dashboard

**Question**: Create a simple dashboard function that displays the current time, weather for a city, and a welcome message. Format it nicely with borders.

현재 시간, 도시의 날씨, 환영 메시지를 표시하는 간단한 대시보드 함수를 만드세요. 테두리로 깔끔하게 형식화하세요.

**Write your answer here (답을 여기에 작성하세요)**:

In [ ]:
# Your code here

---

## References (참고)

1. **Python Error Handling**: https://docs.python.org/3/tutorial/errors.html
2. **Working with JSON**: https://docs.python.org/3/library/json.html
3. **DateTime in Python**: https://docs.python.org/3/library/datetime.html

---

## Key Points (핵심 포인트)

### Remember (기억하세요)
1. **Start simple**: Begin with basic functions, then combine them (간단하게 시작: 기본 함수로 시작해서 결합하기)
2. **Handle errors**: Use try-except to prevent crashes (오류 처리: try-except를 사용해서 충돌 방지)
3. **Save data**: Don't ask for the same information repeatedly (데이터 저장: 같은 정보를 반복해서 요청하지 말기)
4. **Make it pretty**: Good formatting makes information easier to read (예쁘게 만들기: 좋은 형식이 정보를 읽기 쉽게 만듦)

### Final Project Success Tips (최종 프로젝트 성공 팁)
- Test each part separately before combining (결합하기 전에 각 부분을 별도로 테스트)
- Add error handling from the beginning (처음부터 오류 처리 추가)
- Use clear, descriptive variable names (명확하고 설명적인 변수명 사용)
- Start with simple features, add complexity gradually (간단한 기능으로 시작해서 점진적으로 복잡성 추가)

### Congratulations! (축하합니다!)
You've learned how to:
다음을 배웠습니다:
- Write basic Python programs (기본 Python 프로그램 작성)
- Work with data (lists, dictionaries, JSON) (데이터 작업)
- Create visualizations with matplotlib (matplotlib으로 시각화 생성)
- Scrape websites and use APIs (웹사이트 스크래핑 및 API 사용)
- Build complete applications (완전한 애플리케이션 구축)

**You're now ready to create your own Python projects!**
**이제 자신만의 Python 프로젝트를 만들 준비가 되었습니다!**